In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from util import *
import surprise

In [2]:
data = surprise.Dataset.load_builtin('ml-100k')

In [3]:
res = get_fold_matrices(data)

NameError: name 'get_fold_matrices' is not defined

In [ ]:
fold1 = res[0]

### Importing w/ Aaron's fn

In [ ]:
data = surprise.Dataset.load_builtin('ml-100k')
full_tr = data.build_full_trainset()
folds = get_xy(data, full_tr)

In [ ]:
fold1 = folds[0]

In [ ]:
X_train,y_train,X_test,y_test = fold1

In [ ]:
# reshape:
X_train = [X_train[:, 0], X_train[:, 1]]
X_test = [X_test[:, 0], X_test[:, 1]]

In [ ]:
X_train.shape

(80000, 2)

In [ ]:
num_users = np.size(np.unique(X_train[0]))
num_movies = np.size(np.unique(X_train[1]))

In [ ]:
print(num_users)
print(num_movies)

943
1652


In [ ]:
max_users = np.max([np.max(X_train[0]), np.max(X_test[0])])+1
max_movies = np.max([np.max(X_train[1]), np.max(X_test[1])])+1

In [ ]:
print(max_users)
print(max_movies)

943
1682


- 5 folds
- in a fold there are 943 movies

In [ ]:
def NNPredictor(num_users, num_movies, num_factors):
    # two input layers for movies and users
    user_input = keras.layers.Input(shape=(1,))
    movie_input = keras.layers.Input(shape=(1,))

    # two embedding layers for movies and users (transforms indices into dense vectors)
    user_embedding = keras.layers.Reshape((num_factors,))(keras.layers.Embedding(num_users,num_factors,embeddings_initializer='he_normal', embeddings_regularizer = keras.regularizers.l2(1e-16))(user_input))
    movie_embedding = keras.layers.Reshape((num_factors,))(keras.layers.Embedding(num_movies,num_factors,embeddings_initializer='he_normal', embeddings_regularizer = keras.regularizers.l2(1e-16))(movie_input))

    # dense hidden layer
    # ...
    
    # output layer
    out = keras.layers.Dot(axes=1)([user_embedding, movie_embedding])
    
    # Assembling & compiling model
    model = keras.models.Model(inputs = [user_input,movie_input], outputs=out)
    adam = keras.optimizers.Adam(lr=1e-3)
    model.compile(loss='mean_squared_error', optimizer=adam)

    return model

In [ ]:
model = NNPredictor(max_users, max_movies, 50)

In [ ]:
model.fit(x=X_train, y=y_train, batch_size=64, epochs=5, verbose=1) # validation_data=(X_test, y_test)

Epoch 1/5
1250/1250 [==============================] - 2s 1ms/step - loss: 10.3256
Epoch 2/5
1250/1250 [==============================] - 2s 1ms/step - loss: 1.4495
Epoch 3/5
1250/1250 [==============================] - 2s 1ms/step - loss: 0.9543
Epoch 4/5
1250/1250 [==============================] - 2s 1ms/step - loss: 0.8729
Epoch 5/5
1250/1250 [==============================] - 2s 1ms/step - loss: 0.8195


In [ ]:
res = model.predict(x=X_test)

625/625 [==============================] - 1s 1ms/step


In [ ]:
print(type(res))
print(res.shape)

pred = [Prediction(uid, iid, r, est, {}) for (uid, iid, r, est) in zip(X_test[0], X_test[1], y_test, res)]

<class 'numpy.ndarray'>
(20000, 1)


In [ ]:
y_test.size

20000